In [1]:
from Camera import Camera
import numpy as np
from Frame import Frame
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils
from GaussianSplat import GaussianSplat
import Plotters

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1407]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [2]:

input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/output/scaling_lr0.005_percent_dense0.001_densify_grad_threshold0.0002_position_lr_init1.6e-07/point_cloud/iteration_30000/point_cloud.ply"
cam = frames['P1407CAM4.jpg'].get_croped_camera()
gs = GaussianSplat(input_file)  
gs_filtered = gs.filter(gs.projection_filter(frames,gs.xyz,croped_image = True),path = gs.path)
gs_filtered.save_gs()
gs_filtered.calc_cov3d()
gs_filtered.calc_cov2d(cam)


xyz_homo  = np.column_stack((gs_filtered.xyz,np.ones((gs_filtered.xyz.shape[0],1))))


cam.getProjectionMatrix([160,160])
p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel = proj_screen(p_proj[:,:],160)

gs_idx = np.argsort(-pixel[:,2])

gs_filtered.conic = gs_filtered.conic[gs_idx]
gs_filtered.opacity = gs_filtered.opacity[gs_idx]
gs_filtered.xyz = gs_filtered.xyz[gs_idx]
gs_filtered.vertices = gs_filtered.vertices[gs_idx]
gs_filtered.color = gs_filtered.color[gs_idx]


ul,br = gs_filtered.get_rect(cam)






In [12]:
tile_x = 6
tile_y = 7
tile_coords = np.array([tile_x, tile_y])

within_bounds = (ul <= tile_coords) & (br >= tile_coords)
count_within_bounds = np.sum(within_bounds, axis=1) == 2
sum(count_within_bounds)

8851

In [13]:
tile_x = 6
tile_y = 6
tile_coords = np.array([tile_x, tile_y])
pix_start = (tile_coords - 1)*gs_filtered.grid
pix_end = (tile_coords)*gs_filtered.grid


pix_start
pix_end

array([60, 60])

In [45]:
tile_x = 1
tile_y = 2
tile_coords = np.array([tile_x, tile_y])
within_bounds = (ul <= tile_coords) & (br >= tile_coords)
count_within_bounds = np.sum(within_bounds, axis=1) == 2
sum(count_within_bounds)


578

In [3]:
tile = np.zeros((160,160))

for tile_x in range(6,10):
    for tile_y in range(6,10):
        tile_coords = np.array([tile_x, tile_y])

        within_bounds = (ul <= tile_coords) & (br >= tile_coords)
        count_within_bounds = np.sum(within_bounds, axis=1) == 2

        pix_start = (tile_coords - 1)*gs_filtered.grid
        pix_end = (tile_coords)*gs_filtered.grid


        conic = gs_filtered.conic[count_within_bounds]
        color = gs_filtered.color[count_within_bounds]
        opacity = gs_filtered.opacity[count_within_bounds]
        for idx_x,pix_x in enumerate(range(pix_start[0],pix_end[0])):
            for idx_y,pix_y in enumerate(range(pix_start[1],pix_end[1])):
                pixel = [pix_start[0] + idx_x,pix_start[1] + idx_y]
                d = np.column_stack([pixel[0],pixel[1]])
                power = -0.5 * (conic[:,0] * d[:,0] * d[:,0] + conic[:,2] * d[:,1] * d[:,1]) - conic[:,1] * d[:,0] * d[:,1]

                power[power > 0] = 0
                alpha = np.minimum(0.99,  np.exp(power)*1 / (1 + np.exp(-opacity)))
                power[power > 0] = 0
                # alpha = color[:,3]
                # alpha[alpha < 1/255] = 0


                T = 1
                clr = [0,0,0,0]
                for trans,col in zip(alpha,color): 
                    T = T*(1-trans)
                    if T < 0.0001:
                        break
                    clr += col*trans*T

                tile[pixel[0],pixel[1]] = clr[0]



In [4]:
conic

array([], shape=(0, 4), dtype=float64)

In [68]:
T = 1- alpha
plt.plot(T)

In [71]:
plt.imshow(tile)

In [61]:
clr

array([0., 0., 0., 0.], dtype=float32)

In [6]:
range(int(gs_filtered.grid[0]))

range(0, 10)

In [ ]:
for idx in range(gs_filtered.grid.astype(int))

In [ ]:
x 0 10 10 20 20 30


10*i 10*i+10
0 10  10  20  20  30



In [6]:
np.unique(np.column_stack([ul,br]))

array([[ 6.,  1., 10.,  6.],
       [ 6.,  1., 10.,  7.],
       [ 6.,  1., 10.,  6.],
       ...,
       [ 5.,  1., 10.,  6.],
       [ 0.,  4.,  5.,  9.],
       [ 4.,  1., 10.,  6.]])

In [3]:
gs_filtered.grid

[10.9375, 10.9375]

In [37]:
cam = frames['P1407CAM1.jpg'].get_croped_camera()
xyz_homo  = np.column_stack((gs_filtered.xyz,np.ones((gs_filtered.xyz.shape[0],1))))

cam.getProjectionMatrix([160,160])

p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T

p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel = proj_screen(p_proj[:,:],160)

In [38]:
plt.scatter(pixel[:,0],pixel[:,1],c = pixel[:,2])

In [19]:

xyz_homo  = np.column_stack((gs_filtered.xyz,np.ones((gs_filtered.xyz.shape[0],1))))
p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel = proj_screen(p_proj[:,0:2],160)

xy_up_left_corner = np.minimum(gs_filtered.grid,(np.maximum(0,pixel - gs_filtered.radius) / gs_filtered.block_xy).astype(int))
xy_bot_right_corner = np.minimum(gs_filtered.grid,((np.maximum(0,(pixel + gs_filtered.radius + gs_filtered.block_xy - 1) / gs_filtered.block_xy)))).astype(int)

pixel = cam.project_on_image(gs_filtered.xyz)

plt.scatter(xy_up_left_corner[:,0],xy_up_left_corner[:,1])
plt.scatter(xy_bot_right_corner[:,0],xy_bot_right_corner[:,1])



In [29]:
plt.plot((xy_up_left_corner[:,1] -xy_bot_right_corner[:,1])*(xy_up_left_corner[:,0] -xy_bot_right_corner[:,0]))

In [27]:
wak = cam.project_on_image(xyz_homo)
plt.scatter(pixel[:,0],pixel[:,1])
plt.scatter(wak[:,0],wak[:,1])

In [33]:
gs_filtered.radius

34.0

In [ ]:

        
        grid_x, grid_y = grid  # unpack grid dimensions

        # Calculate min and max bounds of the rectangle
        rect_min_x = min(grid_x, max(0, int((p[0] - max_radius) / BLOCK_X)))
        rect_min_y = min(grid_y, max(0, int((p[1] - max_radius) / BLOCK_Y)))
        
        rect_max_x = min(grid_x, max(0, int((p[0] + max_radius + BLOCK_X - 1) / BLOCK_X)))
        rect_max_y = min(grid_y, max(0, int((p[1] + max_radius + BLOCK_Y - 1) / BLOCK_Y)))
        
        rect_min = (rect_min_x, rect_min_y)
        rect_max = (rect_max_x, rect_max_y)

In [8]:
# Plots


hover_text = [f"Color: {color}" for color in gs.color[:,2]]

fig = go.Figure()
Plotters.scatter3d(fig,gs.xyz,gs.color[:,2],3)
fig.update_traces(hovertext=hover_text, hoverinfo='text')

[Plotters.scatter3d(fig,frames[list(frames.keys())[cam]].voxels_with_idx,'red',3,opa = 0.2) for cam in range(4)]
fig.show()
Plotters.scatter_projections_from_gs(frames,gs, plot_image = False)

fig = go.Figure()
Plotters.scatter3d(fig,gs_filtered.xyz,gs_filtered.color[:,2],3,opa = 0.5)
fig.update_traces(hovertext=hover_text, hoverinfo='text')
fig.show()
# fig.write_html(f"I:/My Drive/Research/gs_data/mov19_2022_03_03/output/ptcloud.html")
Plotters.scatter_projections_from_gs(frames,gs_filtered, plot_image = False)
#


FileNotFoundError: [Errno 2] No such file or directory: 'I:\\My Drive\\Research\\gs_data\\mov19_2022_03_03\\output\\ptcloud.html'

## Elliptical Gaussian (from EWA article)


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$

To optimize the coordinates of the 3D points, we minimize the reprojection error. The reprojection is calculated using the projection matrix. By utilizing the Jacobian of the projection matrix, we can update the current 3D point in the direction that minimizes the error, following the gradient.


$p$ - the observed 2D point

$\mathbf{P}(\mathbf{X})$ - the projected 2D point 
Calculate the error vector:

$error = \mathbf{p} - \mathbf{P}(\mathbf{X}) $
Update the gradient of the error function:  

$ \nabla E = \mathbf{J}^T \cdot \text{error} $
Update the 3d point:  

$ X_{new} = X_{old} - \alpha\cdot\nabla E $



In [1]:
scale = gs.scale[0]
scale

import numpy as np
mod = 1
S = np.eye(3) * gs_filtered.scale[:,np.newaxis,:]
S

rot = gs_filtered.rot
q = (rot.T / np.linalg.norm(rot,axis =1)).T# r, x, y, z = q
R = np.column_stack(
    [1.0 - 2.0 * (q[:,2] * q[:,2] + q[:,3] * q[:,3]), 2.0 * (q[:,1]  * q[:,2] - q[:,0] * q[:,3] ), 2.0 * (q[:,1] * q[:,3]  + q[:,0] * q[:,2]),
    2.0 * (q[:,1] * q[:,2] + q[:,0] * q[:,3] ), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,3]  * q[:,3] ), 2.0 * (q[:,2] * q[:,3]  - q[:,0] * q[:,1]),
    2.0 * (q[:,1] * q[:,3]  - q[:,0] * q[:,2]), 2.0 * (q[:,2] * q[:,3]  + q[:,0] * q[:,1]), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,2] * q[:,2])])
R = R.reshape(R.shape[0],3,3)
M = S @ R  # Matrix multiplication
Sigma = M.transpose(0,2,1) @ M

image_width = 160
image_height = 160
image = 'P1407CAM1.jpg'
import math
def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames[image].world_to_cam



t = np.matmul(viewmat , np.column_stack((gs_filtered.xyz,np.ones(gs_filtered.xyz.shape[0]))).T).T
limxy = np.column_stack((np.array([1.3 * tan_fovx]*t.shape[0]), np.array([1.3 * tan_fovy]*t.shape[0])))
t[:, :2] = np.minimum(limxy, np.maximum(-limxy, t[:,0:2]/t[:,2:])) 
zero_np = np.zeros((t.shape[0],1))


J = np.column_stack((
    fx / t[:,2:], zero_np, - (fx * t[:,0:1]) / (t[:,2:] ** 2),
    zero_np, fy / t[:,2:], - (fy * t[:,1:2]) / (t[:,2:] ** 2),
    zero_np, zero_np, zero_np
))

J = J.reshape((J.shape[0],3,3))

W = np.tile(viewmat[0:3,0:3].T,(J.shape[0],1,1))
T = W @ J
cov = T.transpose(0,2,1) @ Sigma @ T

cov = np.squeeze(np.dstack((cov[:,0,0],cov[:,0,1],cov[:,1,1])))

NameError: name 'gs' is not defined

In [50]:
np.squeeze(np.dstack((cov[:,0,0],cov[:,0,1],cov[:,1,1])))

(16484, 3)

In [43]:
cov.shape

(16484, 3, 3)

In [29]:
q = (rot.T / np.linalg.norm(rot,axis =1)).T
q

array([[ 0.96445376, -0.08434534, -0.06010166, -0.24311022],
       [ 0.99281603, -0.10408249, -0.04416541, -0.03914728],
       [ 0.9234634 , -0.3006936 , -0.11951439,  0.2061917 ],
       ...,
       [ 0.6595076 ,  0.18798089, -0.72526795,  0.06082167],
       [ 0.74300337,  0.5309335 , -0.07509685,  0.40051985],
       [ 0.7351371 ,  0.14402883,  0.44993064, -0.48620126]],
      dtype=float32)

In [36]:
cov[0]


array([[ 0.27606161, -0.3983559 , -0.00470983],
       [-0.3983559 ,  2.07604314,  0.06530628],
       [-0.00470983,  0.06530628,  0.00236078]])

In [34]:
covs[0][0]

array([ 0.2760616 , -0.39835588,  2.07604308])

In [175]:
bla = R[0].copy()
bla

array([[ 9.9999857e-01, -7.5298308e-06, -9.4716688e-07],
       [ 1.8136902e-06,  9.9998641e-01, -1.9317500e-05],
       [ 7.3692609e-06,  1.7881750e-05,  9.9998659e-01]], dtype=float32)

In [19]:
covs[0]

array([array([ 0.2760616 , -0.39835588,  2.07604308]),
       array([ 5.64345323e-09,  1.76841623e-09, -6.62277013e-10,  3.47003664e-09,
               1.88305455e-10,  1.42008109e-10])                                 ,
       array([ 0.96445376, -0.08434534, -0.06010166, -0.24311022], dtype=float32)],
      dtype=object)

In [10]:
Sigma[0]

NameError: name 'im_name' is not defined

In [65]:
image_width = 160
image_height = 160
image = 'P1407CAM1.jpg'
import math
def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)

viewmat = frames[image].world_to_cam
t = np.matmul(viewmat , np.column_stack((gs_filtered.xyz,np.ones(gs_filtered.xyz.shape[0]))).T).T

limx = np.array([1.3 * tan_fovx]*t.shape[0])
limy = np.array([1.3 * tan_fovy]*t.shape[0])

txtz = t[:,0] / t[:,2]
tytz = t[:,1] / t[:,2]
t[:,0] = np.minimum(limx, np.maximum(-limx, txtz)) * t[:,2]
t[:,1] = np.minimum(limy, np.maximum(-limy, tytz)) * t[:,2]
zero_np = np.zeros((t.shape[0],1))


J = np.column_stack((
    fx / t[:,2:], zero_np, - (fx * t[:,0:1]) / (t[:,2:] ** 2),
    zero_np, fy / t[:,2:], - (fy * t[:,1:2]) / (t[:,2:] ** 2),
    zero_np, zero_np, zero_np
))

J = J.reshape((J.shape[0],3,3))

W = np.tile(viewmat[0:3,0:3].T,(J.shape[0],1,1))
T = W @ J
cov = T.transpose(0,2,1) @ Sigma @ T

# np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (12365,3,3)->(12365,newaxis,newaxis) (16484,3,3)->(16484,newaxis,newaxis)  and requested shape (3,3)

In [13]:
import math

def compute_cov_3d(scale, mod, rot):
    # Create scaling matrix
    S = np.eye(3) * mod * scale

    # Normalize quaternion to get a valid rotation
    q = rot / np.linalg.norm(rot)
    r, x, y, z = q

    # Compute rotation matrix from quaternion
    R = np.array([
        [1.0 - 2.0 * (y * y + z * z), 2.0 * (x * y - r * z), 2.0 * (x * z + r * y)],
        [2.0 * (x * y + r * z), 1.0 - 2.0 * (x * x + z * z), 2.0 * (y * z - r * x)],
        [2.0 * (x * z - r * y), 2.0 * (y * z + r * x), 1.0 - 2.0 * (x * x + y * y)]
    ])

    M = S @ R  # Matrix multiplication

    # Compute 3D world covariance matrix Sigma
    Sigma = M.T @ M
    cov_3d = np.zeros(6, dtype=float)
    # Covariance is symmetric; store upper right
    cov_3d[0] = Sigma[0, 0]
    cov_3d[1] = Sigma[0, 1]
    cov_3d[2] = Sigma[0, 2]
    cov_3d[3] = Sigma[1, 1]
    cov_3d[4] = Sigma[1, 2]
    cov_3d[5] = Sigma[2, 2]

    # Covariance is symmetric; store upper right
    return cov_3d,q

def compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat):

    t = np.matmul(viewmat , np.append(mean,1)).T

 
    limx = 1.3 * tan_fovx
    limy = 1.3 * tan_fovy
    txtz = t[0] / t[2]
    tytz = t[1] / t[2]
    t[0] = min(limx, max(-limx, txtz)) * t[2]
    t[1] = min(limy, max(-limy, tytz)) * t[2]

    # Compute Jacobian matrix J
    J = np.array([
        [fx / t[2], 0.0, - (fx * t[0]) / (t[2] ** 2)],
        [0.0, fy / t[2], - (fy * t[1]) / (t[2] ** 2)],
        [0, 0, 0]
    ])


    W = viewmat[0:3,0:3].T
    # Compute T matrix
    T = W @ J  # Matrix multiplication
    # Construct Vk matrix (3D covariance matrix)
    Vrk = np.array([
        [cov_3d[0], cov_3d[1], cov_3d[2]],
        [cov_3d[1], cov_3d[3], cov_3d[4]],
        [cov_3d[2], cov_3d[4], cov_3d[5]]
    ])

 

    # Compute the covariance matrix
    cov = T.T @ Vrk.T @ T

    # Apply low-pass filter: ensure Gaussian is at least one pixel wide/high
    # cov[0, 0] += 0.3
    # cov[1, 1] += 0.3

    return np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def calc_det(cov):
    return (cov[0] * cov[2] - cov[1] * cov[1])

def calc_conic(cov,det):
    det_inv = 1 / det
    return  [cov[2] * det_inv, -cov[1] * det_inv, cov[0] * det_inv ]

In [8]:

image_width = 160
image_height = 160

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames[image].world_to_cam



input_file = "D:/Documents/gray_dense_grad/output2/position_lr_init_1.6e-05_scaling_lr0.01_dens_iter500_featur0.0025/point_cloud/iteration_15000/point_cloud.ply"
pos = gs_filtered.xyz
color = gs_filtered.color
scale = gs_filtered.scale
rot = gs_filtered.rot



NameError: name 'image' is not defined

In [44]:
viewmat

array([[ 0.48336487, -0.49025624,  0.72526355,  0.00099953],
       [-0.8753999 , -0.27616153,  0.39674907,  0.00096338],
       [ 0.00578118, -0.82667021, -0.56265704,  0.27667279]])

In [17]:

mod = 1.0

idx = -1
mean = pos

def get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy):
    cov_3d,q = compute_cov_3d(scale, 1, rot)
    cov = compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat)
    return cov,cov_3d,q

# det = calc_det(cov)
# conic = calc_conic(cov,det)

# covs = np.vstack([get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy) for scale,rot,mean in zip(scale,rot,mean)])

homo_pos = frames[image].add_homo_coords(pos)
proj = frames[image].project_on_image(homo_pos,croped_camera_matrix = True)


c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [18]:
covs[0]

array([array([ 0.2760616 , -0.39835588,  2.07604308]),
       array([ 5.64345323e-09,  1.76841623e-09, -6.62277013e-10,  3.47003664e-09,
               1.88305455e-10,  1.42008109e-10])                                 ,
       array([ 0.96445376, -0.08434534, -0.06010166, -0.24311022], dtype=float32)],
      dtype=object)

In [10]:

image = 'P1407CAM1.jpg'


pos = gs_filtered.xyz
color = gs_filtered.color
scale = gs_filtered.scale
rot = gs_filtered.rot
indices1 = (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1)
cov = gs_filtered.cov2d
indices = (np.abs(cov[:,1])>1) & (np.abs(cov[:,2])>1) & (np.abs(cov[:,0])>1) & (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1) & (np.abs(cov[:,1])<100) & (np.abs(cov[:,2])<100) & (np.abs(cov[:,0])<100)
for_ell = gs_filtered.cov2d[indices]
homo_pos = frames[image].add_homo_coords(gs_filtered.xyz)
proj = frames[image].project_on_image(homo_pos,croped_camera_matrix = True)


for_ell_proj = proj[indices]
color_ell = gs_filtered.color[indices]

In [71]:
for_ell_proj

array([[-0.00471224, -0.00875376, -0.00031085],
       [-0.00471327, -0.00877849, -0.00027387],
       [-0.0046862 , -0.00846439, -0.00037821],
       ...,
       [-0.00077174, -0.00744362, -0.00139848],
       [-0.00078346, -0.00743715, -0.0013813 ],
       [-0.00430325, -0.00926331, -0.00026157]], dtype=float32)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_gaussian_ellipse(mean, cov_2d, ax=None, n_std=1.0, **kwargs):
    """Plot a 2D Gaussian as an ellipse."""
    if ax is None:
        ax = plt.gca()

    # Covariance matrix as 2x2
    cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                           [cov_2d[1], cov_2d[2]]])

    # Eigenvalues and eigenvectors for the orientation and size
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
    order = eigenvals.argsort()[::-1]
    eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

    # Calculate the angle of the ellipse (in degrees)
    vx, vy = eigenvecs[:, 0]
    theta = np.degrees(np.arctan2(vy, vx))

    # Ellipse parameters
    width, height = 2 * n_std * np.sqrt(eigenvals)
    
    # Create the ellipse
    ell = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    
    ax.add_patch(ell)
    return width, height,theta


# Plot the Gaussian as an ellipse
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0,160)
# plt.scatter(proj[:,0],proj[:,1],s = 1)

# Plot the Gaussian with 1 standard deviation
b = [plot_gaussian_ellipse(for_ell_proj, for_ell, ax=ax, facecolor = color_ell[0:3], alpha=color_ell[3]) for for_ell_proj,for_ell,color_ell in zip(for_ell_proj[0:1500],for_ell[0:1500],color_ell[0:1500])]
# Show the plot
plt.grid(True)
plt.show()

C:\Users\Roni\AppData\Local\Temp\ipykernel_25800\586283624.py:24: RuntimeWarning:

invalid value encountered in sqrt



In [122]:
for_ell[:,0]*for_ell[:,2] - for_ell[:,1]*for_ell[:,1] 

array([[ 1.31638512e+02,  5.60281618e+01,  1.20569344e+02],
       [ 1.73860418e+02, -6.57871162e+01,  7.50580432e+01],
       [ 1.04310354e+02,  3.11110422e+01,  6.51293764e+01],
       ...,
       [ 5.11134414e-01, -1.07170627e-01,  3.54871447e-01],
       [ 4.21098920e+00,  9.85440507e+00,  2.51314231e+01],
       [ 1.18874675e+00,  2.02123437e+00,  4.89711273e+00]])

In [277]:
n_std = 1
cov_2d = for_ell
# Covariance matrix as 2x2
cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                        [cov_2d[1], cov_2d[2]]])

# Eigenvalues and eigenvectors for the orientation and size
eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
order = eigenvals.argsort()[::-1]
eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

# Calculate the angle of the ellipse (in degrees)
vx, vy = eigenvecs[:, 0]
theta = np.degrees(np.arctan2(vy, vx))

# Ellipse parameters
width, height = 2 * n_std * np.sqrt(eigenvals)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [119]:
for_ell_proj

array([[ 84.19078169,  47.06180881],
       [ 76.25174261,  87.23355188],
       [ 84.18230826,  47.46902143],
       ...,
       [ 44.42871227,  63.13493924],
       [117.26919451,  99.38322307],
       [117.19633068,  99.37221166]])

In [275]:
im_name

'P1407CAM1.jpg'

In [281]:
plydata = PlyData.read(input_file)
im_name = list(frames.keys())[0]
color = gs.color
pos = gs.xyz

# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    indices =(color[:,0] < 0.2) & (color[:,1] < 1) &(color[:,2] < 1) & (color[:,3] < 1) & (color[:,0] > 0) & (color[:,1] > 0) &(color[:,2] > 0) 

    splat = pos[indices,:]
    colors = color[indices, :]  # Filtered colors (RGB or RGBA)
    colors[:,3] = colors[:,3]
    homo_voxels_with_idx = frames[image].add_homo_coords(splat[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = colors)
    # axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = 'red')